In [1]:
import os
import string
from pathlib import Path

In [2]:
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)
data_dir = current_dir.joinpath('data')
data_dir.mkdir(parents=True, exist_ok=True)


print(current_dir)
print(results_dir)
print(data_dir)

c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment10
c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment10\results
c:\Users\saman\git_repos\dsc650\dsc650\assignments\assignment10\data


#### Assignment 10.1.a  

Create a tokenize function that splits a sentence into words. Ensure that your tokenizer removes basic punctuation.

In [4]:
file_name = "sample.txt"
file_path = f"{data_dir}\{file_name}"
file_path

'c:\\Users\\saman\\git_repos\\dsc650\\dsc650\\assignments\\assignment10\\data\\sample.txt'

In [13]:
def tokenize(sentence):
    tokens = []
    words = sentence.split()
    # tokenize the sentence
    for word in words:
        tokens.append(word)
    return tokens

In [16]:
sentence = ''' How are you? 
				I am doing fine. Today is Sunday. Time is 9 AM.
			'''
# with open(file_path,'r') as f:
# 	sentence = f.readline()
# 	# lower case letters
# 	sentence = sentence.lower()

# 	# print(contents)
tokenize(sentence)

['How',
 'are',
 'you?',
 'I',
 'am',
 'doing',
 'fine.',
 'Today',
 'is',
 'Sunday.',
 'Time',
 'is',
 '9',
 'AM.']

#### Assignment 10.1.b

Implement an `ngram` function that splits tokens into N-grams. 

In [7]:
def ngram(tokens, n):
    ngrams = []
    # Create ngrams
    return ngrams

#### Assignment 10.1.c

Implement an one_hot_encode function to create a vector from a numerical vector from a list of tokens.

In [8]:
def one_hot_encode(tokens, num_words):
    token_index = {}
    results = ''
    return results